In [239]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pickle

In [240]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [241]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [242]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [243]:
y = df.Survived
X = df.drop('Survived', axis=1)

In [244]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [245]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [246]:
# Let's first start by imputing, since two columns are missing values and these columns are one numeric and other categorical so we apply two imputations
age_si = SimpleImputer()
embarked_si = SimpleImputer(strategy='most_frequent')

# For age (numeric column)
X_train_age_imputed = age_si.fit_transform(X_train[['Age']])
X_test_age_imputed = age_si.transform(X_test[['Age']])

# For embarked (categoric column)
X_train_embarked_imputed = embarked_si.fit_transform(X_train[['Embarked']])
X_test_embarked_imputed = embarked_si.transform(X_test[['Embarked']])

In [247]:
X_train_embarked_imputed.shape

(712, 1)

In [248]:
X_train.shape

(712, 7)

In [249]:
# Now, applying OneHotEncoder for two nominal categorical columns i.e, Sex and Embarked
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first', dtype='int32')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first', dtype='int32')

# Applying OHE For sex columns
X_train_ohe_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_test_ohe_sex = ohe_sex.transform(X_test[['Sex']])

# Applying OHE For Embarked columns
X_train_ohe_embarked = ohe_embarked.fit_transform(X_train[['Embarked']])
X_test_ohe_embarked = ohe_embarked.transform(X_test[['Embarked']])

In [250]:
X_train_age_imputed

array([[29.74518389],
       [31.        ],
       [31.        ],
       [20.        ],
       [21.        ],
       [45.5       ],
       [22.        ],
       [29.74518389],
       [29.74518389],
       [26.        ],
       [25.        ],
       [21.        ],
       [31.        ],
       [15.        ],
       [29.74518389],
       [29.74518389],
       [65.        ],
       [29.74518389],
       [ 1.        ],
       [34.        ],
       [49.        ],
       [18.        ],
       [29.74518389],
       [70.        ],
       [14.        ],
       [19.        ],
       [30.        ],
       [31.        ],
       [32.        ],
       [16.        ],
       [50.        ],
       [24.        ],
       [56.        ],
       [ 7.        ],
       [ 9.        ],
       [33.        ],
       [19.        ],
       [32.5       ],
       [ 1.        ],
       [45.        ],
       [29.74518389],
       [19.        ],
       [21.        ],
       [ 4.        ],
       [28.        ],
       [17

In [251]:
X_train_remaining_cols = X_train.drop(columns=["Age", "Sex", "Embarked"], axis=1)
X_test_remaining_cols = X_test.drop(columns=["Age", "Sex", "Embarked"], axis=1)

In [252]:
X_test_remaining_cols

,Pclass,SibSp,Parch,Fare
495,3,0,0,14.4583
648,3,0,0,7.5500
278,3,4,1,29.1250
31,1,1,0,146.5208
255,3,0,2,15.2458
...,...,...,...,...
780,3,0,0,7.2292
837,3,0,0,8.0500
215,1,1,0,113.2750
833,3,0,0,7.8542


In [253]:
X_train_transformed = np.concatenate((X_train_remaining_cols, X_train_age_imputed, X_train_ohe_sex, X_train_ohe_embarked), axis=1)
X_test_transformed = np.concatenate((X_test_remaining_cols, X_test_age_imputed, X_test_ohe_sex, X_test_ohe_embarked), axis=1)

In [254]:
X_train_transformed.shape

(712, 9)

In [255]:
X_test_transformed.shape

(179, 9)

In [256]:
# Now creating an instance of a model
model = DecisionTreeClassifier()
model.fit(X_train_transformed, y_train)
preds = model.predict(X_test_transformed)
preds

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1], dtype=int64)

In [257]:
acc_score = accuracy_score(y_test, preds)
print(f"Accuracy score of model: {acc_score}")

Accuracy score of model: 0.7821229050279329


In [258]:
# Exporting our model to be used somewhere in which user give inputs like Sex, Embarked, Fare etc, and we will predict whether they survived.
with open('models/ohe_sex.pkl', 'wb') as file:
    pickle.dump(ohe_sex, file)

with open('models/ohe_embarked.pkl', 'wb') as file:
    pickle.dump(ohe_embarked, file)

with open('models/model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [283]:
y_test, preds

(495    0
 648    0
 278    0
 31     1
 255    1
       ..
 780    1
 837    0
 215    1
 833    0
 372    0
 Name: Survived, Length: 179, dtype: int64,
 array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 1], dtype=int64))